In [1]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from multiprocessing import Process
from empath import Empath
import pandas as pd
import numpy as np
import textblob
import spacy
import time
import csv
import re
data_df = pd.read_csv('G:\\sufe\\mental illness\\Dataset\\labeled\\positive\\data\\positive_tweet.csv')
#删除网址  mention 停用词
tweet_text = data_df['tweet']

[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
stopWords = set(stopwords.words('english'))
nlp = spacy.load('en_core_web_sm')
prog = re.compile("(@[A-Za-z0-9]+)|([^0-9A-Za-z' \t])|(\w+:\/\/\S+)")
prog2 = re.compile(" +")
lexicon = Empath()
empath_cols = ["{0}_empath".format(v) for v in lexicon.cats.keys()]

In [3]:
def lemmatization(x, nlp):
    tweets = " ".join(list(x.values))
    letters_only = prog.sub(" ", tweets)
    lemmatized = []
    for token1 in nlp(letters_only):
        if token1.lemma_ != "-PRON-" and token1 not in stopWords:
            lemmatized.append(token1.lemma_)
        else:
            lemmatized.append(token1.text)
    final = prog2.sub(" ", " ".join(lemmatized))
    return final


def empath_analysis(x):
    val = lexicon.analyze(x, normalize=True)
    if val is None:
        return lexicon.analyze(x)
    else:
        return val
x=data_df['tweet'][:20]
#print(lemmatization(x,nlp))

In [49]:
def processing(vals, columns, iterv):
    users = pd.DataFrame(vals)
    users = users[columns]

    print("{0}-------------".format(iterv))

    # PRE-PROCESSING

    users_text = users.groupby(["uid"])["tweet"].apply(lambda x: lemmatization(x, nlp)).reset_index()
    print("{0}-------------PRE-PROCESSING".format(iterv))

    # SENTIMENT ANALYSIS

    sentiment_arr = np.array(list(users_text["tweet"].apply(lambda x: textblob.TextBlob(str(x)).sentiment).values))
    sentiment_cols = ["sentiment", "subjectivity"]
    df_sentiment = pd.DataFrame(sentiment_arr, columns=sentiment_cols, index=users_text.uid.values)
    print("{0}-------------SENTIMENT".format(iterv))

    # EMPATH ANALYSIS

    lexicon_arr = np.array(list(users_text["tweet"].apply(lambda x: empath_analysis(x)).values))
    df_empath = pd.DataFrame.from_records(index=users_text.uid.values, data=lexicon_arr)
    df_empath.columns = empath_cols
    print("{0}-------------EMPATH".format(iterv))

    # MERGE TO SINGLE

    df = pd.DataFrame(pd.concat([df_empath, df_sentiment], axis=1))
    #df.set_index("uid", inplace=True)
    df.to_csv("G:\\sufe\\mental illness\\Dataset\\labeled\\positive\\data\\users_content_{0}.csv".format(iterv))
    print("-------------{0}".format(iterv))
#print(data_df.columns.values.tolist())

In [50]:
f =open('G:\\sufe\\mental illness\\Dataset\\labeled\\positive\\data\\positive_tweet.csv','r',encoding='utf-8')
cols = ['favorite_count', 'favorited', 'id', 'in_reply_to_status_id', 'lang', 'retweet_count', 'retweeted', 'time', 'tweet', 'uid']
csv_dict_reader = csv.DictReader(f)

acc_vals = []
iter_vals, count, count_max, last_u, v = 1, 0, 50000, None, []
last_u=None
for line in csv_dict_reader:
    if last_u is not None and last_u != line["uid"]:
        #提取特征
        s = time.time()
        processing(v, cols, iter_vals)
        print(time.time() - s)        
        count, last_u, v = 0, None, []
        iter_vals += 1
    v.append(line)
    count += 1
    if count >= count_max:
        last_u = line["uid"]

s = time.time()
processing(v, cols, iter_vals)
print(time.time() - s)
v=[]
last_u=None
iter_vals=0

1-------------
1-------------PRE-PROCESSING
1-------------SENTIMENT
1-------------EMPATH
-------------1
174.48697996139526
2-------------
2-------------PRE-PROCESSING
2-------------SENTIMENT
2-------------EMPATH
-------------2
185.12958884239197
3-------------
3-------------PRE-PROCESSING
3-------------SENTIMENT
3-------------EMPATH
-------------3
180.23130869865417
4-------------
4-------------PRE-PROCESSING
4-------------SENTIMENT
4-------------EMPATH
-------------4
179.71827936172485
5-------------
5-------------PRE-PROCESSING
5-------------SENTIMENT
5-------------EMPATH
-------------5
182.392431974411
6-------------
6-------------PRE-PROCESSING
6-------------SENTIMENT
6-------------EMPATH
-------------6
177.95917868614197
7-------------
7-------------PRE-PROCESSING
7-------------SENTIMENT
7-------------EMPATH
-------------7
176.90411853790283
8-------------
8-------------PRE-PROCESSING
8-------------SENTIMENT
8-------------EMPATH
-------------8
180.59932947158813
9-------------
9--

'\ns = time.time()\nprocesses = []\nfor i in acc_vals:\n    p = Process(target=processing, args=(i[0], i[1], i[2]))\n    processes.append(p)\nfor p in processes:\n    p.start()\nfor p in processes:\n    p.join()\nprint(time.time() - s)\nacc_vals = []\n'